<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
موضوع‌بندی
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک تمرین صنعتی از  کاربرد یادگیری ماشین در پردازش زبان طبیعی (NLP) خوش آمدید.  در این تمرین داده‌های واقعی وب فارسی که توسط پلتفرم <a href="https://www.yektanet.com/">یکتانت</a> پالایش و جمع‌آوری شده در اختیار ما قرار گرفته است. هدف تمرین؛ ساخت یک مدل یادگیری ماشین است که با توجه متن‌های موجود در یک پیوند (Link) نظیر <i>عنوان</i>، <i>توضیحات</i>، <i>محتوای متنی [کامل]</i> و غیره بتواند دسته‌ی موضوعی آن سند را پیش‌بینی کند. به‌عنوان مثال اگر پیوندی از یک سایت خبری با عنوان «<i>کیهان کلهر جایزه مرد سال موسیقی جهان را دریافت کرد</i>» داشته باشیم، مدل ما باید پیش‌بینی کند که این مطلب مرتبط با موضوع «موسیقی» است. البته در این مثال ما تنها از ویژگی <i>عنوان</i> یاد کردیم، در حالی‌که می‌توان از متن <i>توضیحات</i> یا <i>محتوای متنی</i> هم کمک گرفت. 

</br>
توجه داشته باشید برای آن‌که بتوانید از الگوریتم‌های معرفی‌شده در کالج جهت کار روی داده‌های متنی استفاده کنید نیاز است حداقل با یکی از روش‌های تعبیه‌سازی (Embedding) آشنا شده باشید تا بتوانید هر متن را به یک بردار ویژگی عددی تبدیل کنید.

</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [369]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from hazm import *

import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


import scipy.sparse as ss

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    هر نمونه از این مجموعه‌داده با ویژگی‌هایی که در جدول زیر شرح داده شده همراه است. ستون <code>category</code> متغیر هدف مسئله است که موضوع محتوا را نشان می‌دهد. 
</font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>| موضوع (متغیر هدف) |
|<code>description</code>| توضیحات |
|<code>text_content</code>| محتوای متنی |
|<code>title</code>| عنوان |
|<code>h1</code>| محتوای تگ <code>h1</code> صفحه |
|<code>h2</code>|محتوای تگ <code>h2</code> صفحه  |
|<code>url</code>| آدرس پیوند|
|<code>domain</code>|دامنه‌ی وب‌سایت |
|<code>id</code>|آیدی پیوند|

</font>
</div>
</center>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>yektanet_train.csv</code> و نمونه‌های آزمون که باید موضوع آن‌ها را پیش‌بینی کنید در فایل <code>yektanet_test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از مجموعه‌ی آموزشی را به عنوان مجموعه‌ی اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [370]:
train = pd.read_csv('../data/yektanet_train.csv')
train

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...,عکس بیت‌کوین کش برای پروفایل,عکس بیت‌کوین کش برای پروفایل,عکس بیت کوین با کیفیت 4K عکس ارزهای دیجیتال عک...,jowhareh.com/photo/%D8%B9%DA%A9%D8%B3-%D8%A8%D...,jowhareh.com,3268
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی,آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات...,doctor-yab.ir/Search/14773/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,175
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...,دانلود تحریم‌گذر Geph برای اندروید,دانلود تحریم‌گذر Geph برای اندروید,دانلود نرم افزار Geph,palexe.site/dl/geph-android/,palexe.site,3402
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ,راه‌اندازی تلویزیون همگام‌سازی کنترل اتصال به ...,rokhdadeghtesadi.ir/43874/,rokhdadeghtesadi.ir,3811
...,...,...,...,...,...,...,...,...,...
4784,سلامت,سلام خانم دکتر یه سوال مهم دارم آیا قرص ال دی ...,قرص ال دی با دوفاستون تداخل داره؟ پرسش (1397/0...,قرص ال دی با دوفاستون تداخل داره؟,قرص ال دی با دوفاستون تداخل داره؟,NaN,ooma.org/ask/77389/%D9%82%D8%B1%D8%B5-%D8%A7%D...,ooma.org,769
4785,خانه و باغبانی,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...,فروش اقساطی ماشین لباسشویی اسنوا سری هارمونی 8...,فروش اقساطی کالای دیجیتال فروش اقساطی لوازم اد...,ghesticlub.com/Products/Snowa-Harmony-8-kg-was...,ghesticlub.com,1226
4786,غذا و نوشیدنی,قهوه سرا در مشهد -شماره تلفن قهوه سرا در مشهد ...,قهوه درویش در مشهد - پشت بام تهران ورود/ثبت نا...,قهوه درویش در مشهد - پشت بام,قهوه درویش در مشهد,قهوه درویش در مشهد,poshtebam.com/Banners/AdsDetail/377919/,poshtebam.com,2967
4787,ورزش,گزارش برخی رسانه‌های خارجی نشان می‌دهد اطلاعات...,کمک غیرمستقیم فیفا به کی‌روش! | تمام رقیبان تی...,کمک غیرمستقیم فیفا به کی‌روش! | تمام رقیبان تی...,کمک غیرمستقیم فیفا به کی‌روش! | تمام رقیبان تی...,Hamshahri Online برچسب‌ها پر بیننده‌ترین اخبار...,hamshahrionline.ir/news/715355/%DA%A9%D9%85%DA...,hamshahrionline.ir,2684


In [371]:
test = pd.read_csv('../data/yektanet_test.csv')
test

,description,text_content,title,h1,h2,url,domain,id
0,پیام ناشناس یک ابزار رایگان برای ساخت چالش های...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,NaN,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,abzarek.ir/service-p/msg/824757/,abzarek.ir,3291
1,اقتصادنیوز: سید امیر احمد علیه السلام ملقب به ...,حرم شاهچراغ شیراز در گذر زمان+تصاویر حرم شاهچر...,حرم شاهچراغ شیراز در گذر زمان+تصاویر,حرم شاهچراغ شیراز در گذر زمان+تصاویر,NaN,eghtesadnews.com/%D8%A8%D8%AE%D8%B4-%D8%A7%D8%...,eghtesadnews.com,5053
2,آخرین خبر های بین المللی و آخرین تحولات جهان، ...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد ✖...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد,زلنسکی پیام پوتین را دریافت کرد,NaN,akharinkhabar.ir/world/9332333/%D8%B2%D9%84%D9...,akharinkhabar.ir,564
3,آهنگ گاهی دریدن لازم نیست با صدای امیر تتلو با...,آهنگ گاهی دریدن لازم نیست امیر تتلو آهنگ گاهی ...,آهنگ گاهی دریدن لازم نیست امیر تتلو,آهنگ گاهی دریدن لازم نیست امیر تتلو,گاهی دریدن لازم نیست امیر تتلو,behmusic.com/music/%DA%AF%D8%A7%D9%87%DB%8C-%D...,behmusic.com,3607
4,کیهان با اشاره به حوادث و ناآرامیهای اخیر کشور...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,برچسب‌ها نظر شما آخرین اخبار سیاسی ایران در بح...,khabaronline.ir/news/1688132/%D8%A7%D9%86%D8%A...,khabaronline.ir,2358
...,...,...,...,...,...,...,...,...
412,نوبت دهی دکتر حسین کرمی متخصص جراح کلیه و مجار...,دکتر حسین کرمی متخصص جراح کلیه و مجاری ادراری ...,دکتر حسین کرمی متخصص جراح کلیه و مجاری ادراری ...,دکتر حسین کرمی,آدرس و تلفن دکتر حسین کرمی نظرات و تجربیات در ...,doctor-yab.ir/Search/2214/%D8%AF%DA%A9%D8%AA%D...,doctor-yab.ir,67
413,نوبت دهی دکتر مینا اسلام دوست متخصص چشم پزشک د...,دکتر مینا اسلام دوست متخصص چشم پزشک در هشتگرد ...,دکتر مینا اسلام دوست متخصص چشم پزشک در هشتگرد,دکتر مینا اسلام دوست,آدرس و تلفن دکتر مینا اسلام دوست نظرات و تجربی...,doctor-yab.ir/Search/13803/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,1229
414,دانلود فول آلبوم افسر شهیدی شامل با کیفیت 320 ...,دانلود فول آلبوم افسر شهیدی | کاملترین فول آلب...,دانلود فول آلبوم افسر شهیدی | کاملترین فول آلب...,دانلود فول آلبوم افسر شهیدی دانلود فول آلبوم ا...,دانلود فول آلبوم افسر شهیدی قسمتی از بیوگرافی ...,muhamamusic11.com/album/download-full-album-af...,muhamamusic11.com,42
415,عکس حیوانات زیبا و بامزه برای پروفایل ، عکس حی...,عکس حیوانات زیبا و بامزه برای پروفایل + عکس ها...,عکس حیوانات زیبا و بامزه برای پروفایل + عکس ها...,عکس حیوانات زیبا و بامزه برای پروفایل + عکس ها...,عکس حیوانات زیبا و بامزه برای پروفایل عکس های ...,namagard.com/ax-profile-animal-8/,namagard.com,102


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام کار با داده‌های متنی، پیش‌پردازش داده‌ها به کمک تکنیک‌های پردازش زبان طبیعی یکی از مراحل بسیار تاثیرگذار در یادگیری مدل و عملکرد نهایی است.
در تمرین «کامنت‌کاوی» فصل دسته‌بندی با چندین تکنیک رایج پیش‌پردازش خصوصاً برای زبان فارسی آشنا شدید. در این قسمت نیز می‌توانید تابعی بنویسید که یک رشته (<code>string</code>) را گرفته، اصلاحات موردنظر شما رو روی متن اعمال کرده و در نهایت نتیجه را با فرمت یک رشته (<code>string</code>) خروجی دهد. بررسی و تحلیل کلمات موجود در مجموعه‌داده از نظر تعداد رخداد نیز می‌تواند شما را در پیش‌پردازش بهتر یاری کند.
</font>
</p>


In [372]:
train.describe(include='all')

,category,description,text_content,title,h1,h2,url,domain,id
count,4789,4743,4789,4789,4431,3350,4789,4784,4789.000000
unique,22,4398,4720,4614,4246,2489,4786,827,NaN
top,سلامت,آخرین خبر های ورزشی و فوتبالی لیگ برتر، اروپا ...,ورود | فلای تودی ورود یا ثبت نام × ورود به حسا...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,با پولم چه ماشینی بخرم؟,افزودن به کالکشن,kermany.com/does-eating-coffee-endanger-the-he...,doctor-yab.ir,NaN
freq,614,22,10,17,39,171,2,242,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2602.844644
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1503.061732
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1302.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2601.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3915.000000


In [373]:
# do some preprocessing :)
train.nunique()

category          22
description     4398
text_content    4720
title           4614
h1              4246
h2              2489
url             4786
domain           827
id              4789
dtype: int64

In [374]:
nan_index_train = train[train.isna().any(axis=1)].index
nan_index_train
nan_index_test = test[test.isna().any(axis=1)].index
nan_index_test

Int64Index([  0,   1,   2,   8,  11,  13,  15,  19,  24,  25,
            ...
            385, 386, 391, 392, 395, 398, 399, 403, 405, 409],
           dtype='int64', length=145)

In [375]:
train.iloc[nan_index_train,1] = train.iloc[nan_index_train,3]
test.iloc[nan_index_test,0] = test.iloc[nan_index_test,2]

In [376]:
train.drop(columns=['text_content',	'title',	'h1',	'h2',	'url',	'id'],inplace=True)
test.drop(columns=['text_content',	'title',	'h1',	'h2',	'url',	'id'],inplace=True)
train

,category,description,domain
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",jadvalyab.ir
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,jowhareh.com
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,doctor-yab.ir
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,palexe.site
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,rokhdadeghtesadi.ir
...,...,...,...
4784,سلامت,قرص ال دی با دوفاستون تداخل داره؟,ooma.org
4785,خانه و باغبانی,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...,ghesticlub.com
4786,غذا و نوشیدنی,قهوه سرا در مشهد -شماره تلفن قهوه سرا در مشهد ...,poshtebam.com
4787,ورزش,گزارش برخی رسانه‌های خارجی نشان می‌دهد اطلاعات...,hamshahrionline.ir


In [377]:
train['domain'].fillna('doctor-yab.ir', inplace=True)
test['domain'].fillna('doctor-yab.ir', inplace=True)


In [378]:
train.isna().sum()

category       0
description    0
domain         0
dtype: int64

In [379]:
test.isna().sum()

description    0
domain         0
dtype: int64

In [380]:
train

,category,description,domain
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",jadvalyab.ir
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,jowhareh.com
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,doctor-yab.ir
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,palexe.site
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,rokhdadeghtesadi.ir
...,...,...,...
4784,سلامت,قرص ال دی با دوفاستون تداخل داره؟,ooma.org
4785,خانه و باغبانی,فروش نقدی و اقساطی ماشین لباسشویی اسنوا سری ها...,ghesticlub.com
4786,غذا و نوشیدنی,قهوه سرا در مشهد -شماره تلفن قهوه سرا در مشهد ...,poshtebam.com
4787,ورزش,گزارش برخی رسانه‌های خارجی نشان می‌دهد اطلاعات...,hamshahrionline.ir


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در هنگام تعبیه‌سازی به کمک کتابخانه‌ی <code>scikit-learn</code>
از آنجا که ممکن است توکن‌ساز (Tokenizer) مورد استفاده در این کتابخانه مناسب زبان فارسی نباشد، بهتر است از توکن‌سازهای مناسب این زبان استفاده کنیم. کافیست تابعی بنویسید که یک رشته (<code>string</code>) را گرفته و به کمک کتابخانه‌ی موردنظر شما (مثل <code>word_tokenize</code> در کتابخانه‌ی هضم) توکن‌های آن را جدا کند. خروجی تابع لیستی از توکن‌ها خواهد بود.
</font>
</p>


In [381]:
import string
from hazm import stopwords_list
from nltk.tokenize.casual import TweetTokenizer

normalizer = Normalizer(persian_numbers =False,persian_style=False)
tokenizer = WordTokenizer(separate_emoji=True , replace_links=True ,replace_ids=True ,replace_emails=False ,replace_numbers=True ,replace_hashtags=True)
stop_words_EN = set(stopwords.words('english'))
stop_words_FA = set(stopwords_list())
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
t = TweetTokenizer()

# 
A = -1
def pre_fast(text):
	

	
	#lowercase_text(text)
	text = text.lower()

	text = re.sub(r'\d+', '', text)

	#remove_punctuation(text):
	text = text.translate(str.maketrans('', '', string.punctuation))

	punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~؟!.،,?'''

	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, "")

	punc = '''\u200d♂️٪'🏻','''
	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, "")

	text = normalizer.normalize(text)


	# Remove_stop_words_English and Remove_stop_words_Farsi and Lemmatizer_Farsi and Stemmer_English
	#romove number


	text = re.sub(r'\d+', '', text)
	words = tokenizer.tokenize(text)
	text = ' '.join(words)
	words = t.tokenize(text)


	
	# words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	text = ' '.join(words)




	punc = '''\u200c'''

	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, " ")

	
	text = re.sub(r'\d+', '', text)
	words = tokenizer.tokenize(text)
	text = ' '.join(words)
	words = t.tokenize(text)
	words = t.tokenize(text)
	# words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	text = ' '.join(words)

	text = normalizer.normalize(text)
	#remove_punctuation(text):
	text = text.translate(str.maketrans('', '', string.punctuation))

	text = text.replace('''\u200c''', " ")
	text = text.replace('\u200c', " ")
	
	global A
	print(A)
	A +=1

	return	text



In [382]:
from sklearn.feature_extraction.text import TfidfVectorizer


corpus =  train['description']
tfidf_vect_fullname = TfidfVectorizer(preprocessor =pre_fast,min_df = 4)
final_counts_fullname = tfidf_vect_fullname.fit_transform(corpus)

print(tfidf_vect_fullname.get_feature_names_out())
print(final_counts_fullname.toarray()[0])
print(final_counts_fullname.toarray()[1])

-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276

In [383]:
tfidf_vect_fullname.get_feature_names_out()

array(['amd', 'bodi', 'bبهترین', ..., 'یکم', 'یکپارچه', 'یگان'],
      dtype=object)

In [384]:

test_biography = test['description']
test_counts_fullname = tfidf_vect_fullname.transform(test_biography)


4788
4789
4790
4791
4792
4793
4794
4795
4796
4797
4798
4799
4800
4801
4802
4803
4804


4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4821
4822
4823
4824
4825
4826
4827
4828
4829
4830
4831
4832
4833
4834
4835
4836
4837
4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856
4857
4858
4859
4860
4861
4862
4863
4864
4865
4866
4867
4868
4869
4870
4871
4872
4873
4874
4875
4876
4877
4878
4879
4880
4881
4882
4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
4921
4922
4923
4924
4925
4926
4927
4928
4929
4930
4931
4932
4933
4934
4935
4936
4937
4938
4939
4940
4941
4942
4943
4944
4945
4946
4947
4948
4949
4950
4951
4952
4953
4954
4955
4956
4957
4958
4959
4960
4961
4962
4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4981
4982
4983
4984
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999
5000
5001
5002
5003
5004


In [385]:
final_counts_fullname.toarray().shape

(4789, 2907)

In [386]:
test_counts_fullname.toarray().shape

(417, 2907)

In [387]:
train_data = pd.DataFrame(final_counts_fullname.toarray())
test_data = pd.DataFrame(test_counts_fullname.toarray())
train_data

,0,1,2,3,4,5,6,7,8,9,...,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [388]:
train.drop(columns= ['description'],inplace=True)
test.drop(columns= ['description'],inplace=True)

train

,category,domain
0,کتاب و ادبیات,jadvalyab.ir
1,تجارت و اقتصاد,jowhareh.com
2,سلامت,doctor-yab.ir
3,تکنولوژی و کامپبوتر,palexe.site
4,تکنولوژی و کامپبوتر,rokhdadeghtesadi.ir
...,...,...
4784,سلامت,ooma.org
4785,خانه و باغبانی,ghesticlub.com
4786,غذا و نوشیدنی,poshtebam.com
4787,ورزش,hamshahrionline.ir


In [389]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['category'] = le.fit_transform(train['category'])
to_label = dict(zip(le.classes_, le.transform(le.classes_)))
to_class = dict(zip(le.transform(le.classes_), le.classes_))
print(to_label)
train

{'اجتماعی': 0, 'اشتغال': 1, 'تجارت و اقتصاد': 2, 'تحصیلات': 3, 'تکنولوژی و کامپبوتر': 4, 'حقوق و دولت و سیاست': 5, 'حیوانات خانگی': 6, 'خانه و باغبانی': 7, 'خانواده': 8, 'خودرو': 9, 'سفر و گردشگری': 10, 'سلامت': 11, 'علم و دانش': 12, 'غذا و نوشیدنی': 13, 'فیلم و سینما': 14, 'مد و زیبایی': 15, 'مذهبی': 16, 'مسکن': 17, 'موسیقی': 18, 'هنر و سرگرمی': 19, 'ورزش': 20, 'کتاب و ادبیات': 21}


,category,domain
0,21,jadvalyab.ir
1,2,jowhareh.com
2,11,doctor-yab.ir
3,4,palexe.site
4,4,rokhdadeghtesadi.ir
...,...,...
4784,11,ooma.org
4785,7,ghesticlub.com
4786,13,poshtebam.com
4787,20,hamshahrionline.ir


In [390]:
from sklearn.preprocessing import OrdinalEncoder

categorical_cols = train.select_dtypes(include = "object").columns

enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

train[categorical_cols] = enc.fit_transform(train[categorical_cols])
test[categorical_cols] = enc.transform(test[categorical_cols])
test

,domain
0,16.0
1,213.0
2,29.0
3,104.0
4,401.0
...,...
412,193.0
413,193.0
414,-1.0
415,523.0


In [391]:
concatenated_test_data= pd.concat([test, test_data],axis=1)
concatenated_test_data.columns = concatenated_test_data.columns.astype(str)
concatenated_test_data

,domain,0,1,2,3,4,5,6,7,8,...,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906
0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,401.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,193.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
413,193.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
414,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.32439,0.0,0.0,0.0,0.0
415,523.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0


In [392]:
concatenated_train_data = pd.concat([train, train_data],axis=1)
concatenated_train_data.columns = concatenated_train_data.columns.astype(str)
concatenated_train_data

,category,domain,0,1,2,3,4,5,6,7,...,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906
0,21,356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,372.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,193.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,566.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,633.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4784,11,558.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4785,7,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4786,13,604.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4787,20,299.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید انتخاب این‌که از کدام ویژگی‌ها (متن‌ها) به‌عنوان ورودی الگوریتم استفاده کنید بر عهده‌ی خودتان است. می‌توانید تنها از یک ستون مثل <code>title</code> استفاده کنید یا می‌توانید به‌نحوی متن یا حتی بردار ویژگی هر ستون را با همدیگر ترکیب کنید. همچنین فراموش نکنید که ستون متغیر هدف یعنی <code>category</code> نیاز به کدگذاری دارد.
</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون وقت آن رسیده که الگوریتم یادگیری ماشین موردنظر خود را بر روی داده‌های آموزشی اجرا کنید. در انتخاب الگوریتم کاملاً آزاد هستید. برای این بخش می‌توانید ابتدا هر ورودی متن را به کمک تکنیک‌های معرفی‌شده در درسنامه‌های این فصل (مثل Bag-of-Word یا Tf-idf) به بردارهای ویژگی عددی تبدیل کنید. سپس این بردارها را همراه با لیست برچسب‌های صحیح به الگوریتم بدهید تا مدل آموخته شود. علاوه بر این می‌توانید تمام این مراحل را در یک خط لوله‌ (<code>Pipeline</code>) نیز تعریف کنید.
</font>
</p>

In [393]:
from sklearn.model_selection import train_test_split
X = concatenated_train_data.drop(columns=['category'])
y = concatenated_train_data['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [394]:
# modeling :)
from sklearn.neural_network import MLPClassifier



model = MLPClassifier()

model.fit(X_train, y_train)


MLPClassifier()

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>F1-score</code> نام دارد و از رویکرد میانگین‌گیری وزن‌دار (<code dir=ltr>average='weighted'</code>) استفاده می‌شود.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید و طبق نتایج به‌دست‌آمده پارامترهای مدل خود را بهتر تنظیم کنید.
</font>
</p>

In [395]:
y_pred = model.predict(X_test)

In [396]:
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report, f1_score

f1 = f1_score(y_test, y_pred, average='weighted') 

print("F1 Score:", f1)

class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

F1 Score: 0.7388310875426325
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.44      0.48        39
           1       0.50      0.20      0.29         5
           2       0.86      0.62      0.72        50
           3       1.00      0.38      0.56        13
           4       0.60      0.81      0.69        32
           5       0.67      0.75      0.70        75
           6       0.50      0.33      0.40         3
           7       0.65      0.71      0.68        21
           8       0.25      0.25      0.25         4
           9       0.82      0.70      0.76        44
          10       0.66      0.90      0.76        30
          11       0.85      0.79      0.82        91
          12       0.00      0.00      0.00         4
          13       0.78      0.84      0.81        25
          14       0.93      0.78      0.85        36
          15       0.33      0.67      0.44        12
          16       0.74      

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدل‌سازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در مجموعه‌ی آزمون استفاده کنید و نتایج را در یک دیتافریم تک‌ستونه با نام <code>submission</code> و در قالب زیر آماده کنید. توجه داشته باشید که ترتیب پیش‌بینی شما اهمیت دارد یعنی به عنوان مثال پیش‌بینی مدل برای نمونه‌ی آزمون <code>m</code> ام را باید در ردیف <code>m</code> ام این دیتافریم ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>|پیش‌بینی مدل شما (از جنس رشته)|
    
</font>
</div>
</center>

In [397]:
pred = model.predict(concatenated_test_data)
pred

array([19,  5,  5, 18,  5,  9, 18, 18,  9,  1, 19,  9,  9,  5, 11,  5,  4,
       20,  5,  5, 11,  4, 13,  7, 20, 11, 20,  2, 14,  2, 13, 11, 15, 17,
       20, 10, 15, 16,  7, 19,  5,  2,  9, 11,  9,  5, 18, 16, 20, 18, 15,
       16, 20, 11, 10,  7, 18, 18, 11, 16,  2, 20, 10, 19,  2,  9, 15,  4,
        7,  2, 20, 14, 19, 18, 19, 11,  5, 11, 18, 20,  9,  5, 10,  3,  5,
        9, 20,  5,  5, 11,  0, 20, 21, 14, 15,  5, 20, 19,  5, 21, 20,  5,
       18,  5, 16, 19, 19, 18,  2,  4, 20, 18, 11, 11, 19,  2, 11, 13,  5,
        0, 17, 10, 19,  5,  9,  4,  2, 17, 10,  4,  5,  4,  4, 17,  5, 20,
        5, 11, 10,  9,  2, 18,  5, 11,  5,  0, 13, 20,  5,  9,  5,  4, 11,
       13,  5,  3,  2,  5,  3, 20,  9, 20,  5, 13,  1, 11,  2,  9, 19, 13,
        5,  4, 21, 20,  0, 11, 11, 11, 11,  0, 19, 11,  5, 13,  5, 10, 15,
       19, 20, 14, 17,  5,  5,  5, 10,  4,  5, 11, 20, 19, 20, 18, 19,  5,
        9, 11, 20, 17, 14, 20, 16, 20, 11,  9, 21, 18, 17,  5, 16, 19,  9,
        9, 15, 10, 19, 19

In [398]:
submission = pd.DataFrame(pred,columns=['category'])
submission

,category
0,19
1,5
2,5
3,18
4,5
...,...
412,11
413,11
414,18
415,19


In [415]:
original_dict = {'a': 'b'}

swapped_dict = {value: key for key, value in to_label.items()}

print(swapped_dict)


{0: 'اجتماعی', 1: 'اشتغال', 2: 'تجارت و اقتصاد', 3: 'تحصیلات', 4: 'تکنولوژی و کامپبوتر', 5: 'حقوق و دولت و سیاست', 6: 'حیوانات خانگی', 7: 'خانه و باغبانی', 8: 'خانواده', 9: 'خودرو', 10: 'سفر و گردشگری', 11: 'سلامت', 12: 'علم و دانش', 13: 'غذا و نوشیدنی', 14: 'فیلم و سینما', 15: 'مد و زیبایی', 16: 'مذهبی', 17: 'مسکن', 18: 'موسیقی', 19: 'هنر و سرگرمی', 20: 'ورزش', 21: 'کتاب و ادبیات'}


In [417]:
submission['category'].replace(swapped_dict,inplace=True)
submission

,category
0,هنر و سرگرمی
1,حقوق و دولت و سیاست
2,حقوق و دولت و سیاست
3,موسیقی
4,حقوق و دولت و سیاست
...,...
412,سلامت
413,سلامت
414,موسیقی
415,هنر و سرگرمی


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [418]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['text_categorization.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['text_categorization.ipynb', 'submission.csv']
